In [4]:
import sys
sys.path.append('SimuQ')
%load_ext autoreload
%autoreload 2
import numpy as np
from simuq import QSystem
from simuq import qubit

from simuq.solver import generate_as
from simuq.aais.ibm import generate_qmachine

from qiskit_ibm_provider import IBMProvider
from qiskit import QuantumCircuit

backend = IBMProvider().get_backend('ibmq_guadalupe')

mach = generate_qmachine(backend)

def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    if p==3:    
        parameter_list = np.array(
        [
            0.5702193 * 2,
            -0.58631086,
            0.85160685 * 2,
            -1.7058538,
            0.29468536 * 2,
            -1.132814,
        ]
    )/2
    if p==2:
        parameter_list = np.array([-3.8034,1.2438, -1.2467,-2.4899])/2
    if p==1:
        parameter_list = np.array([-0.7854,-2.3562])/2

    for i in range(p):
       
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].X * ql[q1].X
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i+1] * ql[q0].Z
        qs.add_evolution(h, 1)
    return qs

def calc_cut(count):
    cut=0
    n_shots=sum(count.values())
    for key in count:
        cut+=count[key]*calc_weight(key)/n_shots
    return cut
def calc_weight(key):
    cut=0
    for i in range(len(key)):
        if key[i]!= key[(i+1)%len(key)]:
            cut+=1
    return cut
# mach=GenMach(12)
qs=GenQS(12,1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from qiskit_ionq import IonQProvider

backend = provider.get_backend("ionq_qpu.aria-1", gateset="qis")

In [ ]:
for i in range(1, 4):
    count = backend.retrieve_job(job_id_dict["cnot"][i]).result().get_counts()
    print("cnot, p=", i)
    print(calc_cut(count))
    count = backend.retrieve_job(job_id_dict["native"][i]).result().get_counts()
    print("native, p=", i)
    print(calc_cut(count))

In [8]:
from simuq.backends.ionq_qiskit_qis_qaoa import transpile
from qiskit import transpile as transpile_qiskit
from qiskit import Aer

circ = transpile(*generate_as(qs, mach))
circ = transpile_qiskit(circ, basis_gates=["rz", "rx", "cx", "id"])
backend = provider.get_backend("ionq_qpu.aria-1", gateset="qis")
# backend=provider.get_backend("ionq_simulator",gateset="qis")

job = backend.run(circ)
print(job.job_id())

# count = job.result().get_counts()
# calc_cut(count)

IonQRetriableError: IonQRetriableError('An unexpected error occurred')

In [16]:
from backends.ionq_qiskit_native_qaoa import transpile
from qiskit import Aer

circ = transpile(*generate_as(qs, mach))

backend = provider.get_backend("ionq_qpu.aria-1", gateset="native")
# backend=provider.get_backend("ionq_simulator",gateset="native")
job = backend.run(circ)
print(job.job_id())
# count = job.result().get_counts()
# calc_cut(count)

ae8f03e6-2dcd-4776-bbec-dc58e31f51b8


6.591796875

In [25]:
with open("native_qaoa.json", "w") as f:
    f.write(qiskit_to_ionq(circ, backend, passed_args={"shots": 1000}))

In [ ]:
backend = provider.get_backend("ionq_qpu.aria-1", gateset="native")
circ = qiskit_to_ionq(circ, backend=backend, passed_args={"shots": 4000})
with open("native_circ.json", "w") as outfile:
    outfile.write(circ)

In [40]:
from backends.ionq_qiskit_qis import transpile

backend = provider.get_backend("ionq_qpu.aria-1", gateset="qis")
circ = transpile(*generate_as(qs, mach))

from qiskit import transpile as transpile_qiskit

circ = transpile_qiskit(circ, basis_gates=["rz", "rx", "id", "cx"])
circ = qiskit_to_ionq(circ, backend=backend, passed_args={"shots": 4000})
with open("cnot_circ.json", "w") as outfile:
    outfile.write(circ)

In [19]:
qs = GenQS()

from qiskit import Aer

backend = Aer.get_backend("qasm_simulator")
from backends.ionq_qiskit_qis import transpile

circ = transpile(*generate_as(qs, mach))
# backend = provider.get_backend("ionq_simulator",gateset="qis")
job = backend.run(circ)
count = job.result().get_counts()